In [ ]:
import tifffile
import torch
from pathlib import Path 
import plotly.express as px
import numpy as np
from postprocessing import apply_flow

ROOT_DIR = Path("../")
IMAGE_ID = "0791"

In [ ]:
mask = tifffile.imread(ROOT_DIR / f"train/a_GT/man_seg{IMAGE_ID}.tif") != 0

flow = tifffile.imread(ROOT_DIR / f"train/a_flow/man_seg{IMAGE_ID}_flow.tif")
flow = torch.tensor(flow).unsqueeze(0).repeat(16, 1, 1, 1)
flow = flow.permute(0, 3, 1, 2)  # from NHWC to NCHW

In [ ]:
positions = apply_flow(flow)

In [ ]:
px.scatter(x=positions[15, :, :, 1][mask], y=positions[15, :, :, 0][mask]).update_yaxes(
    scaleanchor="x",
    scaleratio=1,
).update_layout(width=800, height=800)

In [ ]:

batch_size = mask_gradients.shape[0]
image_height = mask_gradients.shape[2]
image_width = mask_gradients.shape[3]

# positions in (batch_size, image_height, image_width, 2)
positions = (
    torch.cartesian_prod(
        torch.arange(image_height),
        torch.arange(image_width),
    )
    .reshape(image_height, image_width, 2)
    .unsqueeze(0)
    .repeat(batch_size, 1, 1, 1)
).double()

In [ ]:
mask.shape

In [ ]:
positions /= (
    torch.tensor([image_height, image_width]).double().unsqueeze(0).unsqueeze(0)
)
positions = positions * 2 - 1

mask_gradients *= 2.0 / torch.tensor([image_height, image_width]).unsqueeze(-1).unsqueeze(-1)

In [ ]:
positions.shape

In [ ]:
mask_gradients.shape

In [ ]:
positions = positions[:, :, :, [1, 0]]
mask_gradients = mask_gradients[:, [1, 0], :, :]
for _ in range(100):
    gradients = torch.nn.functional.grid_sample(
        mask_gradients, positions, align_corners=False
    )
    for k in range(2):
        positions[:, :, :, k] = positions[:, :, :, k] + gradients[:, k, :, :]
torch.clamp(positions, -1, 1)
positions = positions[:, :, :, [1, 0]]
mask_gradients = mask_gradients[:, [1, 0], :, :]

In [ ]:
positions.shape

In [ ]:
positions = (positions + 1) / 2
positions *= (
    torch.tensor([image_height, image_width]).double().unsqueeze(0).unsqueeze(0)
)
px.scatter(x=positions[0, :, :, 1][mask], y=positions[0, :, :, 0][mask]).update_yaxes(
    scaleanchor="x",
    scaleratio=1,
).update_layout(width=800, height=800)